## 1. 반자동 문서 스캐너

### 마우스 이벤트 등록

In [ ]:
import cv2

def mouse_handler(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # 마우스 왼쪽 버튼 누름
        print('왼쪽 버튼 down')
        print(x, y)
    elif event == cv2.EVENT_LBUTTONUP:  # 마우스 왼쪽 버튼 뗌
        print('왼쪽 버튼 Up')
        print(x, y) 

img = cv2.imread('poker.jpg')
cv2.namedWindow('Test') # Test라는 이름의 윈도우를 먼저 만들어 둠. 여기에 마우스 이벤트를 처리하기 위한 핸들러 적용
cv2.setMouseCallback('Test', mouse_handler)

cv2.imshow('Test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 프로젝트

In [2]:
import cv2
import numpy as np

point_lst = []
src_img = cv2.imread('poker.jpg')

COLOR = (255, 0, 255)   # 핑크
THICKNESS = 3
drawing = False # 선을 그릴지 여부 판단


def mouse_handler(event, x, y, flags, param):
    global drawing
    dst_img = src_img.copy()

    if event == cv2.EVENT_LBUTTONDOWN:  # 마우스 왼쪽 버튼 누름
        drawing = True # 선을 그리기 시작
        point_lst.append((x, y))
    
    if drawing:
        prev_point = None   # 직선의 시작점
        for point in point_lst:
            cv2.circle(dst_img, point, 10, COLOR, cv2.FILLED)
            if prev_point:
                cv2.line(dst_img, prev_point, point, COLOR, THICKNESS, cv2.LINE_AA)
            prev_point = point
        
        next_point = (x, y)

        if len(point_lst) == 4:
            show_result()   # 결과 출력
            next_point = point_lst[0]   # 마지막 점과 맨 첫번째 점을 연결해줌

        cv2.line(dst_img, prev_point, next_point, COLOR, THICKNESS, cv2.LINE_AA)
    
    cv2.imshow('Test', dst_img)

def show_result():
    width, height = 530, 710

    src = np.float32(point_lst)
    dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)  # output 4개 지점

    matrix = cv2.getPerspectiveTransform(src, dst)  # src영역을 dst영역으로 바꾸기 위한 변환행렬 얻어옴
    result = cv2.warpPerspective(src_img, matrix, (width, height))  # 변환행렬을 적용하여 최종 결과이미지를 얻어옴
    cv2.imshow('Result', result)
    

cv2.namedWindow('Test') # Test라는 이름의 윈도우를 먼저 만들어 둠. 여기에 마우스 이벤트를 처리하기 위한 핸들러 적용
cv2.setMouseCallback('Test', mouse_handler)

cv2.imshow('Test', src_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2. 개별 카드 추출해서 파일 저장

In [2]:
import cv2
img = cv2.imread('card.png')    # 원본
target_img = img.copy() # 사본
COLOR = (0, 200, 0) # 녹색
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
ret, otsu = cv2.threshold(gray, -1, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU)

contrs, hierarchy = cv2.findContours(otsu, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

idx = 0
for cnt in contrs:
    if cv2.contourArea(cnt) > 25000:    # 경계선 면적이 25,000 이상인 윤곽선만 그리고 싶음
        x, y, width, height = cv2.boundingRect(cnt) # 윤곽선을 둘러싸고 있는 사각형의 정보 4개를 반환
        cv2.rectangle(target_img, (x, y), (x+width, y+height), COLOR, 2)
        
        # 그린 영역을 잘라서 파일 저장
        crop = img[y:y+height, x:x+width]
        cv2.imshow(f'Card_Crop_{idx+1}', crop)
        # cv2.imwrite(f'Card_Crop_{idx+1}.png', crop)   # 파일 저장
        idx += 1

cv2.imshow('Contour', target_img)
cv2.waitKey(0)
cv2.destroyAllWindows()